In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc
import plot as plot
import slices as slices
from ionfrac import VMIN, VMAX
# import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    snaps[ii].load_shock_variables()
    snaps[ii].load_fluxes()

In [ ]:
vmin_density, vmax_density = -22.1, -14.7    # 1 Density Slice
vmin_zvelocity, vmax_zvelocity = 0, 3.5      # 2 z-Velocity Slice
vmin_temp, vmax_temp = 0, 7                  # 3 Temperature Slice
vmin_ionfrac, vmax_ionfrac = -10, 0          # 4 Ionization Fraction Slice
vmin_emis, vmax_emis = -20, 0                # 5 Emissivity Slice
vmin_iontemp, vmax_iontemp = 0, 7            # 6 Ionized Gas Temperature Projection
slicescales = np.array([4000,25000])
nu = 5.3*10**9

def slice_images(Snapshot, year_str, savefolder = slices.SAVELOC, 
                 show_cbar=False, xlabel=False, cbar_pad=.15):
    # density slice
    file0 = slices.density_slice_pcolormesh(Snap=Snapshot, year=year_str, scale = scale, 
                             vmin = vmin_density, vmax=vmax_density, saveloc=savefolder, 
                             show=True, cmap='cividis', show_cbar=show_cbar, 
                             show_xlabels=xlabel, show_ylabels=True,
                             cbar_pad=cbar_pad)

    # velocity_z slice
    file1 = slices.zvelocity_slice_pcolormesh(Snap=Snapshot, year=year_str, scale = scale, 
                               vmin = vmin_zvelocity, vmax=vmax_zvelocity, saveloc=savefolder, 
                               show=True, show_cbar=show_cbar, 
                               show_xlabels=xlabel, show_ylabels=False,
                               cbar_pad=cbar_pad)

    # temperature slice
    file2 = slices.temp_slice_pcolormesh(Snap=Snapshot, year=year_str, scale = scale, 
                          vmin = vmin_temp, vmax=vmax_temp, saveloc=savefolder, 
                          show=True, cmap='hot', show_cbar=show_cbar, 
                          show_xlabels=xlabel, show_ylabels=False,
                          cbar_pad=cbar_pad)

    # ionization fraction slice
    file3 = slices.ionfrac_slice_pcolormesh(Snap=Snapshot, year=year_str, scale = scale, 
                             vmin = vmin_ionfrac, vmax=vmax_ionfrac, saveloc=savefolder, 
                             show=True, cmap='viridis', show_cbar=show_cbar, 
                             show_xlabels=xlabel, show_ylabels=False,
                             cbar_pad=cbar_pad)

    # emissivity slice
    file4 = slices.emis_slice_pcolormesh(Snap=Snapshot, year=year_str, scale = scale, 
                          vmin = vmin_emis, vmax=vmax_emis, saveloc=savefolder, 
                          show=True, cmap='magma', show_cbar=show_cbar, 
                          show_xlabels=xlabel, show_ylabels=False,
                             cbar_pad=cbar_pad)
    
    return file0, file1, file2, file3, file4

# Colorbars

In [ ]:
def save_slice_cbar(cname, savename, extracrop=30):
    cbimg = Image.open(cname)
    width, height = cbimg.size
    left = 0
    right = width
    top = width+extracrop
    bottom = height
    cbimg = cbimg.crop((left, top, right, bottom))
    cbimg.save(savename)
    return cbimg

In [ ]:
s=1
saveloc = slices.SAVELOC
scale = 25000

# density
cname = slices.density_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_density, vmax=vmax_density, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_0_density.png', extracrop=30)
display(cbimg)

# z velocity
cname = slices.zvelocity_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_zvelocity, vmax=vmax_zvelocity, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_1_zvelocity.png', extracrop=38)
display(cbimg)


# temp
cname = slices.temp_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_temp, vmax=vmax_temp, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_2_temp.png', extracrop=38)
display(cbimg)

# ionization fraction
cname = slices.ionfrac_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_ionfrac, vmax=vmax_ionfrac, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_3_ionfrac.png', extracrop=5)
display(cbimg)

# emissivity
cname = slices.emis_slice_pcolormesh(snaps[s], years[s], scale, 
                             vmin=vmin_emis, vmax=vmax_emis, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_4_emissivity.png', extracrop=18)
display(cbimg)

# PIL Image Methods

In [ ]:
# for PIL    
def make_slices_allimage(imgfiles_arr, scale):
    nrows = len(imgfiles_arr)
    ncols = len(imgfiles_arr[0]) # should be 5 for 
    side=300
    header=0
    if(scale==25000):
        leftax = 150
    else:
        leftax=134
    bottomax = 82
    allimage=Image.new("RGBA", (ncols*side + leftax, 
                                nrows*side + bottomax + header))
    cropped_names = np.empty((nrows, ncols), dtype=object)

    for r in range(nrows):
        for c in range(ncols):
            img = Image.open(imgfiles_arr[r,c])
            width, height = img.size
#             print(width,height)
            if(c==0): # y axis
                new_width=int(side*((width-6.25)/(231-6.25-7.25)))
                left=0
                new_x=0
                right=width-6.25
            else: 
                new_width=side
                left=7.25
                new_x=c*side+(leftax)
                right=width-6.25

            if(r==nrows-1): # x axis
                new_height=int(side*(height-6.75)/(231-6.75-6.75))
                bottom=height
            else: 
                new_height=side
                bottom=height-6.75
            top=7
            new_y = r*side + header

            img = img.crop((left, top, right, bottom))
            img = img.resize((new_width,new_height))
            allimage.paste(img, ((new_x, new_y)))
    display(allimage)
    
    im0 = Image.open(VICO_loc+'/PaperImages/slice_images/cbar_0_density.png')
    im1 = Image.open(VICO_loc+'/PaperImages/slice_images/cbar_1_zvelocity.png')
    im2 = Image.open(VICO_loc+'/PaperImages/slice_images/cbar_2_temp.png')
    im3 = Image.open(VICO_loc+'/PaperImages/slice_images/cbar_3_ionfrac.png')
    im4 = Image.open(VICO_loc+'/PaperImages/slice_images/cbar_4_emissivity.png')
    cbar_ims = np.array([im0, im1, im2, im3, im4])
    cbar_height = 0
    
    for i in range(len(cbar_ims)):
        im = cbar_ims[i]
        im = im.resize((side, int(im.height * side/im.width)))
        if(im.height > cbar_height): cbar_height = im.height
        cbar_ims[i] = im
    imnew = Image.new('RGBA', (allimage.width, allimage.height+cbar_height),
                 color='white')
    imnew.paste(allimage, (0,0))
    for i in range(len(cbar_ims)):
        imnew.paste(cbar_ims[i], (leftax+side*i, allimage.height))
    
    # display(imnew)
    # imnew.save(allimagename)
    return imnew

##### 1 PIL Figure: 25000 au Slices

In [ ]:
nrows = len(snaps)-1
ncols = 5
scale = 25000
slicefiles_25000au = np.empty((nrows, ncols), dtype=object)
r=0
saveloc = VICO_loc+'/pillowfiles/slices_fig'
for s in range(1,len(snaps)):
    # print(years[s])
    if(s < len(snaps)-1):
        slice_images(snaps[s], masss[s], savefolder=saveloc,
                     show_cbar=False, cbar_pad=.15, xlabel=False)
    else:      
        slice_images(snaps[s], masss[s], xlabel=True,
                    show_cbar=False, cbar_pad=.3)
    slicefiles_25000au[r,0] = (saveloc+'/'+snaps[s].name+
                               '_0_density_'+str(scale)+'AU.png')
    slicefiles_25000au[r,1] = (saveloc+'/'+snaps[s].name+
                               '_1_zvelocity_'+str(scale)+'AU.png')
    slicefiles_25000au[r,2] = (saveloc+'/'+snaps[s].name+
                               '_2_temp_'+str(scale)+'AU.png')
    slicefiles_25000au[r,3] = (saveloc+'/'+snaps[s].name+
                               '_3_ionfrac_'+str(scale)+'AU.png')
    slicefiles_25000au[r,4] = (saveloc+'/'+snaps[s].name+
                               '_4_emissivity_'+str(scale)+'AU.png')
    r+=1

In [ ]:
print(f"{slicefiles_25000au.shape=}")
allimage, details = plot.pil_image(slicefiles_25000au)
allimage = plot.pil_slices_cbar(allimage, details, saveloc)
allimage.save(VICO_loc+'/figures/slices/slices_25000au.png')
display(allimage)

##### 2 PIL Figure: 4000 au Slices

In [ ]:
s=1
saveloc = slices.SAVELOC
scale = 4000

# density
cname = slices.density_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_density, vmax=vmax_density, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_0_density.png', extracrop=22)
display(cbimg)

# z velocity
cname = slices.zvelocity_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_zvelocity, vmax=vmax_zvelocity, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_1_zvelocity.png', extracrop=32)
display(cbimg)


# temp
cname = slices.temp_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_temp, vmax=vmax_temp, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_2_temp.png', extracrop=26)
display(cbimg)

# ionization fraction
cname = slices.ionfrac_slice_pcolormesh(snaps[s], years[s], scale, 
                         vmin=vmin_ionfrac, vmax=vmax_ionfrac, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_3_ionfrac.png', extracrop=7)
display(cbimg)

# emissivity
cname = slices.emis_slice_pcolormesh(snaps[s], years[s], scale, 
                             vmin=vmin_emis, vmax=vmax_emis, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc)
cbimg = save_slice_cbar(cname, savename=saveloc+'/cbar_4_emissivity.png', extracrop=6)
display(cbimg)

In [ ]:
nrows = len(snaps)-1
ncols = 5
scale = 4000
slicefiles = np.empty((nrows, ncols), dtype=object)
r=0
saveloc = VICO_loc+'/pillowfiles/slices_fig'
for s in range(1,len(snaps)):
    # print(years[s])
    if(s < len(snaps)-1):
        slice_images(snaps[s], masss[s], savefolder=saveloc,
                     show_cbar=False, cbar_pad=.15, xlabel=False)
    else:      
        slice_images(snaps[s], masss[s], xlabel=True,
                    show_cbar=False, cbar_pad=.3)
    slicefiles[r,0] = (saveloc+'/'+snaps[s].name+
                               '_0_density_'+str(scale)+'AU.png')
    slicefiles[r,1] = (saveloc+'/'+snaps[s].name+
                               '_1_zvelocity_'+str(scale)+'AU.png')
    slicefiles[r,2] = (saveloc+'/'+snaps[s].name+
                               '_2_temp_'+str(scale)+'AU.png')
    slicefiles[r,3] = (saveloc+'/'+snaps[s].name+
                               '_3_ionfrac_'+str(scale)+'AU.png')
    slicefiles[r,4] = (saveloc+'/'+snaps[s].name+
                               '_4_emissivity_'+str(scale)+'AU.png')
    r+=1

slicefiles_4000au = slicefiles

In [ ]:
print(f"{slicefiles_4000au.shape=}")
allimage, details = plot.pil_image(slicefiles_4000au)
allimage = plot.pil_slices_cbar(allimage, details, saveloc)
display(allimage)
allimage.save(VICO_loc+'/figures/slices/slices_4000au.png')